In [1]:
#Based on the following works:
#https://gist.github.com/nariaki3551
#Some data and fonction are confidential so please contact me on email for more details.

In [2]:
import networkx as nx
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.max_open_warning': 0})
import matplotlib.ticker as ticker
from collections import deque, defaultdict
from math import log2, floor, log
from networkx.drawing.nx_agraph import to_agraph 
from copy import deepcopy
from heapq import heappush, heappop
from IPython.display import Image, display_png
from pprint import pprint
import json
from datetime import datetime

In [4]:
#class Node
#def read data
#def read_nodes

In [5]:
MultiGraph=nx.MultiDiGraph()
list_of_nodes = read_nodes('nodes.json')
res_full,TYPES = read_data('full.json')
for i in list_of_nodes:
    MultiGraph.add_node(i.id,lat=i.lat,lon=i.lon)
for i in res_full:
        MultiGraph.add_edge(i[0],i[1],weight=i[2],gender=i[3],key=i[4],spec=i[5])
pos = nx.spring_layout(MultiGraph) 

source = 192
target = 100
K = 10

In [6]:
class Node:
    def __init__(self, name, delta, parent, ix):
        self.name   = name
        self.value  = delta
        self.parent = parent
        self.ix     = ix
        self.left   = None
        self.right  = None
        self.h_out  = None


    def __str__(self):
        return f"name: {self.name}, value: {self.value}, ix: {self.ix}"

def traverse_node(node):
    if node:
        yield node
        for x in traverse_node(node.left):
            yield x
        for x in traverse_node(node.right):
            yield x
        for x in traverse_node(node.h_out):
            yield x

class BinaryTree:
    def __init__(self):
        self.root = None
        self.num  = 0 

    def search(self, x):
        return node.search(self.root, x)

    def insert(self, name, delta):
        if self.root is None:
            self.root = Node(name, delta, None, 0)
        else:
            node_ix = self.num
            dircs = []
            while node_ix > 0:
                dircs.append(node_ix%2) 
                node_ix = (node_ix-1) // 2
           
            curr_node = self.root
            for dirc in dircs[:0:-1]: 
                if dirc:
                    curr_node = curr_node.left
                else:
                    curr_node = curr_node.right
            if dircs[0]:
                curr_node.left  = Node(name, delta, curr_node, self.num)
                curr_node = curr_node.left
            else:
                curr_node.right = Node(name, delta, curr_node, self.num)
                curr_node = curr_node.right

           
            for dirc in dircs[::-1]:
                if curr_node.parent is None\
                or curr_node.value >= curr_node.parent.value:
                    break
               
                parent = curr_node.parent
              
                if parent.parent is not None:
                    if parent.ix % 2:
                        parent.parent.left  = curr_node
                    else:
                        parent.parent.right = curr_node
                if parent.parent is not None:
                    curr_node.parent   = parent.parent
                else:
                    curr_node.parent   = None
                    self.root = curr_node
                parent.parent = curr_node

                parent_right_child = parent.right
                parent_left_child  = parent.left
                parent.right  = curr_node.right
                parent.left   = curr_node.left
              
                if curr_node.right is not None:
                    curr_node.right.parent = parent
                if curr_node.left is not None:
                    curr_node.left.parent  = parent
             
                if curr_node.ix % 2:
                    curr_node.right  = parent_right_child
                    curr_node.left   = parent
                    if parent_right_child is not None:
                        parent_right_child.parent = curr_node
                else:
                    curr_node.left  = parent_left_child
                    curr_node.right = parent
                    if parent_left_child is not None:
                        parent_left_child.parent = curr_node
                parent.ix, curr_node.ix = curr_node.ix, parent.ix
        self.num = self.num + 1

    def root_insert(self, name, delta):
        root_node = Node(name, delta, None, 0)
        if self.root is not None:
            root_node.left = self.root
            self.root.parent = root_node
        self.root = root_node
        self.num = self.num + 1

    def h_out_insert(self, other_tree):
        if self.root is None or other_tree.root is None:
            return
        for node in self.traverse():
            if node.name == other_tree.root.name:
                break
        node.h_out = other_tree.root.left
        if other_tree.root.left is not None:
            other_tree.root.left.parent = node

    def traverse(self):
        if self.root is not None:
            for x in traverse_node(self.root):
                yield x

    
        
if __name__ == 'BinaryTree()':
    a = BinaryTree()

In [7]:
# STEP0 (shrink source - target path)
reachable_nodes_from_source = set()
for edge in nx.bfs_edges(MultiGraph, source=source):
    tail, head = edge
    reachable_nodes_from_source |= {tail, head}
if target not in reachable_nodes_from_source:
    print('There does not exist source {source} - target {target} path')
reachable_nodes_to_target = set()
for edge in nx.bfs_edges(MultiGraph.reverse(), source=target):
    tail, head = edge
    reachable_nodes_to_target |= {tail, head}
MultiGraph = MultiGraph.subgraph(reachable_nodes_from_source & reachable_nodes_to_target)

for tail, head, data in MultiGraph.edges(data=True):
    data['label'] = data['weight']
    
pred, distance = nx.dijkstra_predecessor_and_distance(MultiGraph.reverse(), source=target)

# shortest path tree
T = nx.MultiDiGraph()
for tail, heads in pred.items():
    if heads:
        head = heads[0]
        min_key = min(MultiGraph[tail][head], key=lambda key: MultiGraph[tail][head][key]['weight'])
        weight = MultiGraph[tail][head][min_key]['weight']
        T.add_edge(tail, head, weight=weight, key=min_key)
        
for tail, head, data in T.edges(data=True):
    data['label'] = data['weight']
    
for tail in MultiGraph:
    for head, edges in MultiGraph[tail].items():
        for key in edges:
            edges[key]['delta'] = edges[key]['weight'] + distance[head] - distance[tail]
            
def out(v, MultiGraph, T):
    out_edges = []
    for head, edges in MultiGraph[v].items():
        for key in edges:
            if head not in T[v] or key not in T[v][head]:
                out_edges.append((v, head, key))
    return out_edges

# generate H_out tree
for v in MultiGraph:
    out_edges = sorted(out(v, MultiGraph, T))
    h_out_tree = BinaryTree()
    if out_edges:
        max_root, *other_edges = out_edges
        for edge in other_edges:
            h_out_tree.insert(edge, MultiGraph.edges[edge]['delta'])
        h_out_tree.insert(max_root, MultiGraph.edges[max_root]['delta'])
    MultiGraph.node[v]['H_out'] = h_out_tree

# generate H_T tree
h_t_tree = BinaryTree()
out_root = MultiGraph.node[target]['H_out'].root
if out_root is not None:
    h_t_tree.insert(out_root.name, out_root.value)
MultiGraph.node[target]['H_T'] = h_t_tree
for edge in nx.bfs_edges(T.reverse(), source=target):
    tail, head = edge
    h_t_tree = deepcopy(MultiGraph.node[tail]['H_T'])
    out_root = MultiGraph.node[head]['H_out'].root
    if out_root is not None:
        h_t_tree.insert(out_root.name, out_root.value)
    MultiGraph.node[head]['H_T'] = h_t_tree
    
# generate H_G tree
h_g_tree = deepcopy(MultiGraph.node[target]['H_T'])
h_g_tree.h_out_insert(MultiGraph.node[target]['H_out'])
MultiGraph.node[target]['H_G'] = h_g_tree
for edge in nx.bfs_edges(T.reverse(), source=target):
    _, child = edge
    h_g_tree = MultiGraph.node[child]['H_T']
    for v in nx.shortest_path(T, source=child, target=target):
        h_g_tree.h_out_insert(MultiGraph.node[v]['H_out'])
    MultiGraph.node[child]['H_G'] = h_g_tree

# generate path-graph P
def edge2delta(MultiGraph, edge):
    tail, head, key = edge
    return MultiGraph[tail][head][key]['delta']
def add_H_G_to_P(P, node):
    for v in MultiGraph.node[node]['H_G'].traverse():
        v_name = (node, v.name)
        P.add_node(v_name)
        if v.left  is not None:
            u_name = (node, v.left.name)
            weight = edge2delta(MultiGraph, v.left.name) - edge2delta(MultiGraph, v.name)
            P.add_edge(v_name, u_name, weight=weight, edge_type='heap_edge')
        if v.right is not None:
            u_name = (node, v.right.name)
            weight = edge2delta(MultiGraph, v.right.name) - edge2delta(MultiGraph, v.name)
            P.add_edge(v_name, u_name, weight=weight, edge_type='heap_edge')
        if v.h_out is not None:
            u_name = (node, v.h_out.name)
            weight = edge2delta(MultiGraph, v.h_out.name) - edge2delta(MultiGraph, v.name)
            P.add_edge(v_name, u_name, weight=weight, edge_type='heap_edge')
    return P

## add heap edges
P = nx.DiGraph()    
P = add_H_G_to_P(P, target)
for edge in nx.bfs_edges(T.reverse(), source=target):
    _, child = edge
    P = add_H_G_to_P(P, child)
    
## add cross edges
for v in P:
    _, edge = v
    tail, head, key = edge
    if head not in T[tail] or key not in T[tail][head]:
        _, w, key = edge
        if MultiGraph.node[w]['H_G'].root is not None:
            u_name = (w, MultiGraph.node[w]['H_G'].root.name)
            weight = edge2delta(MultiGraph, MultiGraph.node[w]['H_G'].root.name)
            P.add_edge(v, u_name, weight=weight, edge_type='cross_edge')

## add root node
P.add_node('RootNode')
u = (source, MultiGraph.node[source]['H_G'].root.name)
weight = edge2delta(MultiGraph, MultiGraph.node[source]['H_G'].root.name)
P.add_edge('RootNode', u, weight=weight, edge_type='root_edge')

path_graph_pos = nx.shell_layout(P)
# path_graph_pos = nx.spring_layout(P)
labels = {v: v[1] for v in P}
labels['RootNode'] = 'RootNode'

remain_edges = set()
for edge in nx.bfs_edges(P, source='RootNode'):
    remain_edges.add(edge)
    
# find K - shortest path
def sidetracks2path(sidetracks, P, T, source, target, minus_weight):
    tmp_T = T.copy()
    for v, u in sidetracks[1:]:
        if P[v][u]['edge_type'] == 'cross_edge':
            tail, head, key = v[1]
            tmp_T.add_edge(tail, head, key=key, weight=minus_weight)
    if len(sidetracks) > 1:
        _, last_sidetracks = sidetracks[-1]
        tail, head, key = last_sidetracks[1]
        tmp_T.add_edge(tail, head, key=key, weight=minus_weight)
    shortest_path_node = nx.shortest_path(tmp_T, source=source, target=target, weight='weight')
    shortest_path_edge = []
    for tail, head in zip(shortest_path_node, shortest_path_node[1:]):
        min_key = min(tmp_T[tail][head], key=lambda key: tmp_T[tail][head][key]['weight'])
        shortest_path_edge.append((tail, head, min_key))
    return shortest_path_edge

minus_weight = -sum(abs(edge_data['weight']) for tail, head in T.edges() for edge_data in T[tail][head].values())
A = [] 
B = [(0, [(None, 'RootNode')])] 
k = 0
while B and k < K:
    potentials, sidetracks = heappop(B)
    A.append(sidetracks2path(sidetracks, P, T, source, target, minus_weight))
    _, last_sidetracks = sidetracks[-1]
    for v in P[last_sidetracks]:
        new_sidetracks = sidetracks + [(last_sidetracks, v)]
        new_potentials = potentials + P.edges[last_sidetracks, v]['weight']
        heappush(B, (new_potentials, new_sidetracks))
    k = k + 1
    if k == 1:
        shortest_path_length = sum(MultiGraph[v][u][k]['weight'] for v, u, k in A[-1])
    length = shortest_path_length + potentials
    print(A[-1], length)

[(192, 0, 'BUS'), (0, 4, 'BUS'), (4, 133, 'BUS'), (133, 223, 'BUS'), (223, 97, 'BUS'), (97, 100, 'BUS')] 274
[(192, 0, 'BUS'), (0, 4, 'BUS'), (4, 133, 'BUS'), (133, 221, 'BUS'), (221, 97, 'BUS'), (97, 100, 'BUS')] 274
[(192, 0, 'BUS'), (0, 4, 'BUS'), (4, 131, 'BUS'), (131, 223, 'BUS'), (223, 97, 'BUS'), (97, 100, 'BUS')] 274
[(192, 0, 'BUS'), (0, 4, 'BUS'), (4, 131, 'BUS'), (131, 221, 'BUS'), (221, 97, 'BUS'), (97, 100, 'BUS')] 274
[(192, 0, 'BUS'), (0, 4, 'BUS'), (4, 131, 'BUS'), (131, 222, 'BUS'), (222, 97, 'BUS'), (97, 100, 'BUS')] 274
[(192, 0, 'BUS'), (0, 4, 'BUS'), (4, 133, 'BUS'), (133, 222, 'BUS'), (222, 97, 'BUS'), (97, 100, 'BUS')] 274
[(192, 0, 'BUS'), (0, 9, 'BUS'), (9, 266, 'WALK'), (266, 223, 'BUS'), (223, 97, 'BUS'), (97, 100, 'BUS')] 276
[(192, 0, 'BUS'), (0, 9, 'BUS'), (9, 266, 'WALK'), (266, 221, 'BUS'), (221, 97, 'BUS'), (97, 100, 'BUS')] 276
[(192, 0, 'BUS'), (0, 9, 'BUS'), (9, 266, 'WALK'), (266, 222, 'BUS'), (222, 97, 'BUS'), (97, 100, 'BUS')] 276
[(192, 0, 'BUS')